In [ ]:
!pip install kaggle-environments --upgrade -q

from kaggle_environments import make, evaluate

### Since we are being passed only the last two actions we need to maintain our own history to capture past choices and rewards. This little helper class does just that

In [ ]:
class HistoryCollectingAgent():
    def __init__(self):
        self.my_choices = []
        self.my_rewards = []
        self.opponent_choices = []
    
    def __call__(self, obs, conf):
        if obs.lastActions:
            self.my_choices.append(obs.lastActions[obs.agentIndex])
            self.opponent_choices.append(obs.lastActions[1 - obs.agentIndex])
            self.my_rewards.append(obs.reward - sum(self.my_rewards))
        return self.action(obs, conf)
    
    # abstract method to be implemented in inheriting classes
    def action(self, obs, conf): raise NotImplementedError()

### Then every agent you create can inherit from it and have ready access to the past actions. For example the "least pulled" agent looks like this:

In [ ]:
 class LeastUsedAgent(HistoryCollectingAgent):   
    def count_pulls(self, bandit):
        return self.my_choices.count(bandit) + self.opponent_choices.count(bandit)
    
    def action(self, obs, conf):
        return min(range(conf.banditCount), key=self.count_pulls)

### Since we implemented `__call__` you can pass your agent object as a function:

In [ ]:
env = make("mab", debug=True)
my_agent = LeastUsedAgent()
env.run([my_agent, "random"])
env.render(mode="ipython", width=800, height=500)

### To submit it to the competion we need to add an extra snippet since it expects a `def`

In [ ]:
agent = LeastUsedAgent()
def do_action(obs, conf):
    return agent(obs, conf)